In [4]:
import numpy as np
import pandas as pd
from sklearn import datasets
from sklearn.decomposition import PCA
import math
import scipy.linalg as la
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.spatial import distance
import numba
from numba import jit, int32, int64, float32, float64
import timeit
import time
import pstats
from sklearn.preprocessing import StandardScaler


In [5]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [6]:
#iris = sns.load_dataset('iris')

# matrix data
#X = np.array(iris[['sepal_length', 'sepal_width', 'petal_length', 'petal_width']])
#specs = np.array(iris['species'])

In [60]:
X = np.loadtxt("Data/mnist2500_X.txt")
labels = np.loadtxt("Data/mnist2500_labels.txt")
small = X[:1000]
small.shape

(1000, 784)

## Original Code

In [55]:
def squared_euc_dist(X):
    """Calculate squared euclidean distance for all pairs in a data matrix X with d dimensions and n rows.
    Output is a pairwise distance matrix D that is nxn.
    """
    D = distance.squareform(distance.pdist(X, 'sqeuclidean'))
    return D


def p_cond(d_matrix, sigmas):
    """Convert a distances matrix to a matrix of conditional probabilities."""

    sig_2 = np.square(sigmas.reshape((-1, 1)))
    P_cond = np.exp((d_matrix / (2 * sig_2)) - np.max((d_matrix / (2 * sig_2)), axis=1).reshape([-1, 1]))

    # set p_i|i = 0
    np.fill_diagonal(P_cond, 0.)

    P_cond = (P_cond + 1e-10) / (P_cond + 1e-10).sum(axis=1).reshape([-1, 1])

    return P_cond


def binary_search(eval_fn, target, tol=1e-10, max_iter=10000,
                  lower=1e-20, upper=1000.):
    """Perform a binary search over input values to eval_fn.
    # Arguments
        eval_fn: Function that we are optimising over.
        target: Target value we want the function to output.
        tol: Float, once our guess is this close to target, stop.
        max_iter: Integer, maximum num. iterations to search for.
        lower: Float, lower bound of search range.
        upper: Float, upper bound of search range.
    # Returns:
        Float, best input value to function found during search.
    """
    for i in range(max_iter):
        mid = (lower + upper) / 2.
        val = eval_fn(mid)
        if val > target:
            upper = mid
        else:
            lower = mid
        if np.abs(val - target) <= tol:
            break
    return mid


def perp(d_matrix, sigmas):
    """calculate perplexity from distance matrix, sigmas, and conditional probability matrix."""
    P = p_cond(d_matrix, sigmas)
    entropy = -np.sum(P * np.log2(P), axis=1)
    perplexity = 2 ** entropy

    return perplexity


def find_optimal_sigmas(d_matrix, target_perplexity):
    """For each row of distances matrix, find sigma that results
    in target perplexity for that role."""
    sigmas = []
    # For each row of the matrix (each point in our dataset)
    for i in range(d_matrix.shape[0]):
        # Make fn that returns perplexity of this row given sigma
        eval_fn = lambda sigma: \
            perp(d_matrix[i:i + 1, :], np.array(sigma))
        # Binary search over sigmas to achieve target perplexity
        correct_sigma = binary_search(eval_fn, target_perplexity)
        # Append the resulting sigma to our output array
        sigmas.append(correct_sigma)
    return np.array(sigmas)


def q_ij(Y):
    """Calculate joint probabilities over all points given Y, the low-dimensional map of data points. (pg. 2585)"""

    numerator = np.power(1. + (squared_euc_dist(Y)), -1)
    Q = numerator / np.sum(numerator)

    # q_i|i = 0
    np.fill_diagonal(Q, 0.)

    return Q


def p_ij(X, target_perplexity):
    """Calculate joint probabilities in the high dimensional space given data matrix X
    and a target perplexity to find optimal sigmas (pg. 2584).
    """

    d_matrix = -squared_euc_dist(X)

    # optimal sigma for each row of distance matrix
    sigmas = find_optimal_sigmas(d_matrix, target_perplexity)

    # conditional p matrix from optimal sigmas
    p_conditional = p_cond(d_matrix, sigmas)

    # convert conditional P to joint P matrix (pg. 2584)
    n = p_conditional.shape[0]
    p_joint = (p_conditional + p_conditional.T) / (2. * n)

    return p_joint


def grad_C(P, Q, Y):
    """Calculate gradient of cost function (KL) with respect to lower dimensional map points Y (pg. 2586)"""

    pq_diff = (P - Q)[:, :, np.newaxis]

    y_diff = Y[:, np.newaxis, :] - Y[np.newaxis, :, :]

    y_dist = (np.power(1. + (squared_euc_dist(Y)), -1))[:, :, np.newaxis]

    grad = 4. * (pq_diff * y_diff * y_dist).sum(axis=1)

    return grad


def tsne(X, num_iters=1000, perplexity=30, alpha=10, momentum=0.9):
    """Calculate Y, the optimal low-dimensional representation of data matrix X using optimized TSNE.
    Inputs:
        X: data matrix
        num_iters: number of iterations
        perplexity: target perplexity for calculating optimal sigmas for P probability matrix
        alpha: learning rate
        momentum: momentum to speed up gradient descent algorithm
    """

    # Initialize Y
    np.random.seed(0)
    Y = np.random.normal(0, 0.0001, size=(X.shape[0], 2))
    
    P = p_ij(X, perplexity)

    # Initialise past y_t-1 and y_t-2 values (used for momentum)
    Y_tmin2 = Y
    Y_tmin1 = Y

    # gradient descent with momentum
    for i in range(num_iters):
        Q = q_ij(Y)
        grad = grad_C(P, Q, Y)

        # Update Y using momentum (pg. 2587)
        Y = (Y - alpha * grad) + (momentum * (Y_tmin1 - Y_tmin2))

        # update values of y_t-1 and y_t-2
        Y_tmin2 = Y_tmin1
        Y_tmin1 = Y

    return Y


In [36]:
normal = %timeit -o -r3 -n3 tsne(X[:1000, ])

17min 37s ± 2min 19s per loop (mean ± std. dev. of 3 runs, 3 loops each)
7min 16s ± 3min 53s per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [56]:
%prun -q -D tsne.prof tsne(X[:1000, ])
p = pstats.Stats('tsne.prof')
p.print_stats()
pass

 
*** Profile stats marshalled to file 'tsne.prof'. 
Sat Apr 24 00:18:40 2021    tsne.prof

         1910406 function calls in 1163.256 seconds

   Random listing order was used

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
     2001    0.011    0.000    0.022    0.000 {built-in method builtins.any}
        1    0.000    0.000 1163.256 1163.256 {built-in method builtins.exec}
     2001    0.007    0.000    0.007    0.000 {built-in method builtins.getattr}
    63400    0.089    0.000    0.089    0.000 {built-in method builtins.isinstance}
     4002    0.005    0.000    0.005    0.000 {built-in method builtins.len}
     6003    0.008    0.000    0.008    0.000 {method 'lower' of 'str' objects}
     2001    0.004    0.000    0.004    0.000 {method 'startswith' of 'str' objects}
     2001    0.004    0.000    0.004    0.000 {method 'get' of 'dict' objects}
     2001    0.003    0.000    0.003    0.000 {method 'pop' of 'dict' objects}
   117797    0.121    0.000  

## Reformed and added numba JIT

In [38]:
@jit(nopython = True)
def p_ij(d_matrix, perplexity = 40.0, tol = 1e-6):
    """
    Finds P_ij matrix using binary search to find value of sigma_i

    Inputs: d_matrix- np.array of pairwise distance matrix, with a fixed perplexity

    Output: P-ij matrix
    """

    (n, d) = d_matrix.shape

    P = np.zeros((n, d), dtype=np.float64)
    prec_sum = 0.0    
    
    # precision = 1/2sigma^2
    for i in range(n):
        prec_min = -np.inf
        prec_max = np.inf
        prec = 1.0

        # implement binary search for optimal sigmas 
        for j in range(10): # 10 binary search steps
            sum_p = 0.0
            for k in range(d):
                if k != i:
                    P[i, k] = np.exp(-d_matrix[i, k] * prec)
                    sum_p += P[i, k]

            sum_p_distribution = 0.0

            for k in range(d):
                P[i, k] /= (sum_p + 1e-8)
                sum_p_distribution += d_matrix[i, k] * P[i, k]

            # Calculate entropy, H matrix
            H = np.log(sum_p) + prec * sum_p_distribution
            H_diff = H - np.log(perplexity)

            
            # check if entropy is within tolerance
            if np.fabs(H_diff) <= tol:
                break

            if H_diff > 0.0:
                prec_min = prec
                if prec_max == np.inf:
                    prec *= 2.0
                else:
                    prec = (prec + prec_max) / 2.0
            else:
                prec_max = prec
                if prec_min == -np.inf:
                    prec /= 2.0
                else:
                    prec = (prec + prec_min) / 2.0

        prec_sum += prec

    return P


@jit(nopython = True)
def squared_euc_dist(X):
    """Calculate squared euclidean distance for all pairs in a data matrix X with d dimensions and n rows. 
    Output is a pairwise distance matrix D that is nxn.
    """
    
    norms = np.power(X, 2).sum(axis = 1)
    D = np.add(np.add(-2 * np.dot(X, X.T), norms).T, norms)
    
    return D

@jit(nopython = True)
def q_ij(Y):
    """Calculate joint probabilities over all points given Y, the low-dimensional map of data points. (pg. 2585)"""
    
    numerator = np.power(1. + (squared_euc_dist(Y)), -1)
    Q = numerator / np.sum(numerator)
    
    # q_i|i = 0
    np.fill_diagonal(Q, 0.)
    
    return Q

@jit(nopython = True)
def grad_C(P, Q, Y):
    """Estimate the gradient of t-SNE cost with respect to Y."""
    
    pq_diff = np.expand_dims((P - Q), 2)
    
    y_diff = np.expand_dims(Y, 1) - np.expand_dims(Y, 0)
    
    y_dist = np.expand_dims(np.power(1 + squared_euc_dist(Y), -1), 2)

    grad = 4. * (pq_diff * y_diff * y_dist).sum(axis = 1)
    
    return grad


@jit(nopython = True)
def tsne_opt(X, num_iters = 1000, perplexity = 40, alpha = 100, momentum = 0.8):
    """Calculate Y, the optimal low-dimensional representation of data matrix X using optimized TSNE.
    
    Inputs: 
        X: data matrix 
        num_iters: number of iterations 
        perplexity: target perplexity for calculating optimal sigmas for P probability matrix
        alpha: learning rate
        momentum: momentum to speed up gradient descent algorithm 
    """

    # Initialize Y 
    np.random.seed(0)
    Y = np.random.normal(0, 0.0001, size=(X.shape[0], 2))
    D = squared_euc_dist(X)
    P = p_ij(D)
    P = P + np.transpose(P)
    P = P / np.sum(P)

    # Initialise past y_t-1 and y_t-2 values (used for momentum)
    Y_tmin2 = Y
    Y_tmin1 = Y

    # gradient descent with momentum
    for i in range(num_iters):

        Q = q_ij(Y)
        grad = grad_C(P, Q, Y)

        # Update Y using momentum (pg. 2587)
        Y = (Y - (alpha * grad)) +  (momentum * (Y_tmin1 - Y_tmin2))
  
        # update values of y_t-1 and y_t-2
        Y_tmin2 = Y_tmin1
        Y_tmin1 = Y

    return Y



In [40]:
numb = %timeit -o -r3 -n3 tsne_opt(X[:1000, ])

1min 34s ± 1.87 s per loop (mean ± std. dev. of 3 runs, 3 loops each)


## Cythonize

In [13]:
%load_ext cython

The cython extension is already loaded. To reload it, use:
  %reload_ext cython


In [42]:
%%cython -a

from libc cimport math
cimport cython
import numpy as np
cimport numpy as np
from numpy cimport ndarray
from scipy.spatial import distance

cdef extern from "numpy/npy_math.h":
    float NPY_INFINITY

@cython.boundscheck(False)
@cython.wraparound(False)
cdef p_ij_cy(double[:,:] d_matrix, float perplexity = 40.0, float tol = 1e-5):
    """
    Finds P_ij matrix using binary search to find value of sigmas

    Inputs: X- np.array of pairwise distance matrix, fixed perplexity

    Output: P-ij matrix
    """
    cdef int s = 10 
    
    cdef int n = d_matrix.shape[0], d = d_matrix.shape[1]
    
    cdef np.ndarray[np.float64_t, ndim=2] P = np.zeros(
        (n,d), dtype=np.float64)

    cdef float prec_sum = 0.0

    # precision = 1/2sigma^2
    for i in range(n):
        prec_min = -NPY_INFINITY
        prec_max = NPY_INFINITY
        prec = 1.0
        
        # implement binary search for optimal sigmas 
        for j in range(s):
            sum_p = 0.0
            for k in range(d):
                if k != i:
                    P[i, k] = math.exp(-d_matrix[i, k] * prec)
                    sum_p += P[i, k]

            sum_p_distribution = 0.0

            for k in range(d):
                P[i, k] /= sum_p
                sum_p_distribution += d_matrix[i, k] * P[i, k]

            # Calculate entropy, H matrix
            H = np.log(sum_p) + prec * sum_p_distribution
            H_diff = H - np.log(perplexity)

            if math.fabs(H_diff) <= tol:
                break

            if H_diff > 0.0:
                prec_min = prec
                if prec_max == NPY_INFINITY:
                    prec *= 2.0
                else:
                    prec = (prec + prec_max) / 2.0
            else:
                prec_max = prec
                if prec_min == -NPY_INFINITY:
                    prec /= 2.0
                else:
                    prec = (prec + prec_min) / 2.0

        prec_sum += prec

    return P


@cython.boundscheck(False)
@cython.wraparound(False)
cdef squared_euc_dist(double[:,:] X):
    """Calculate squared euclidean distance for all pairs in a data matrix X with d dimensions and n rows. 
    Output is a pairwise distance matrix D that is nxn.
    """
    cdef int n = X.shape[0]
    cdef int d = X.shape[1]
    cdef double diff
    cdef double dist
    cdef double[:, ::1] D = np.empty((n, n), dtype=np.float64)
    
    for i in range(n):
        for j in range(n):
            dist = 0.0
            for k in range(d):
                diff = X[i, k] - X[j, k]
                dist += diff * diff
            D[i, j] = dist
    return np.asarray(D)
    
@cython.boundscheck(False)
@cython.wraparound(False)
cdef q_ij(double[:,::1] Y):
    """Calculate joint probabilities over all points given Y, the low-dimensional map of data points. (pg. 2585)"""
    
    
    cdef int n = Y.shape[0]
    cdef np.ndarray[np.float64_t, ndim=2] Q = np.empty((n, n), dtype=np.float64)
    cdef double[:, ::1] numerator = np.empty((n, n), dtype=np.float64)

    numerator = 1/(1. + (squared_euc_dist(Y)))
    Q = numerator / (np.sum(numerator))
    cdef int m = Q.shape[0]
    cdef int d = Q.shape[1]
    
    # q_i|i = 0
    for i in range(m):
        for j in range(d):
            if i==j:
                Q[i,j] = 0
    
    return Q

@cython.boundscheck(False)
@cython.wraparound(False)
cdef grad_C(np.ndarray[np.float64_t, ndim=2] P, np.ndarray[np.float64_t, ndim=2] Q, double[:,:] Y):
    """Estimate the gradient of t-SNE cost with respect to Y."""
        
    pq_diff = np.expand_dims((P - Q), 2)
    
    y_diff = np.expand_dims(Y, 1) - np.expand_dims(Y, 0)
    
    y_dist = np.expand_dims(np.power(1 + squared_euc_dist(Y), -1),2)
    
    return  4. * (pq_diff * y_diff * y_dist).sum(axis = 1)

@cython.boundscheck(False)
@cython.wraparound(False)
def tsne_opt_cy(double[:,:] X, int num_iters = 1000, int perplexity = 40, int alpha = 100, float momentum = 0.8):
    """Calculate Y, the optimal low-dimensional representation of data matrix X using optimized TSNE.
    
    Inputs: 
        X: data matrix 
        num_iters: number of iterations 
        perplexity: target perplexity for calculating optimal sigmas for P probability matrix
        alpha: learning rate
        momentum: momentum to speed up gradient descent algorithm 
    """

    # Initialize Y 
    np.random.seed(0)
    cdef int n = X.shape[0]
    Y = np.random.normal(0, 0.0001, size=(n, 2))
    D = squared_euc_dist(X)
    cdef np.ndarray[np.float64_t, ndim=2] P = p_ij_cy(D)
    cdef double[:, :] Pt = P.T
    P = P + P.T
    P = P / np.sum(P)

    # Initialise past y_t-1 and y_t-2 values (used for momentum)
    Y_tmin2 = Y
    Y_tmin1 = Y

    # gradient descent with momentum
    for i in range(num_iters):

        Q = q_ij(Y)
        grad = grad_C(P, Q, Y)

        # Update Y using momentum (pg. 2587)
        Y = (Y - (alpha * grad)) +  (momentum * (Y_tmin1 - Y_tmin2))
  
        # update values of y_t-1 and y_t-2
        Y_tmin2 = Y_tmin1
        Y_tmin1 = Y

    return Y

In [43]:
cy = %timeit -o -r3 -n3 tsne_opt_cy(X[:1000, ])

10min 23s ± 56.5 s per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Initialize JIT with PCA

In [47]:
# initialize X by reducing to 50 dimensions using PCA
train = StandardScaler().fit_transform(X[:1000, ])
X_reduce = PCA(n_components=50).fit_transform(train)

In [48]:
X_reduce.shape

(1000, 50)

In [49]:
numb_pca = %timeit -o -r3 -n3 tsne_opt(X_reduce)

<ipython-input-38-989bb46b08a9>:115: NumbaPerformanceWarning: np.dot() is faster on contiguous arrays, called on (array(float64, 2d, A), array(float64, 2d, A))
  D = squared_euc_dist(X)


1min 27s ± 3.26 s per loop (mean ± std. dev. of 3 runs, 3 loops each)


### JIT with loop distance function

In [67]:
@jit(nopython = True)
def p_ij(d_matrix, perplexity = 40.0, tol = 1e-6):
    """
    Finds P_ij matrix using binary search to find value of sigma_i

    Inputs: d_matrix- np.array of pairwise distance matrix, with a fixed perplexity

    Output: P-ij matrix
    """

    (n, d) = d_matrix.shape

    P = np.zeros((n, d), dtype=np.float64)
    prec_sum = 0.0    
    
    # precision = 1/2sigma^2
    for i in range(n):
        prec_min = -np.inf
        prec_max = np.inf
        prec = 1.0

        # implement binary search for optimal sigmas 
        for j in range(10): # 10 binary search steps
            sum_p = 0.0
            for k in range(d):
                if k != i:
                    P[i, k] = np.exp(-d_matrix[i, k] * prec)
                    sum_p += P[i, k]

            sum_p_distribution = 0.0

            for k in range(d):
                P[i, k] /= (sum_p + 1e-8)
                sum_p_distribution += d_matrix[i, k] * P[i, k]

            # Calculate entropy, H matrix
            H = np.log(sum_p) + prec * sum_p_distribution
            H_diff = H - np.log(perplexity)

            
            # check if entropy is within tolerance
            if np.fabs(H_diff) <= tol:
                break

            if H_diff > 0.0:
                prec_min = prec
                if prec_max == np.inf:
                    prec *= 2.0
                else:
                    prec = (prec + prec_max) / 2.0
            else:
                prec_max = prec
                if prec_min == -np.inf:
                    prec /= 2.0
                else:
                    prec = (prec + prec_min) / 2.0

        prec_sum += prec

    return P


@jit(nopython = True)
def squared_euc_dist(X):
    """Calculate squared euclidean distance for all pairs in a data matrix X with d dimensions and n rows. 
    Output is a pairwise distance matrix D that is nxn.
    """
    n = X.shape[0]
    d = X.shape[1]
    D = np.empty((n, n), dtype=np.float64)
    
    for i in range(n):
        for j in range(n):
            dist = 0.0
            for k in range(d):
                diff = X[i, k] - X[j, k]
                dist += diff * diff
            D[i, j] = dist
    return np.asarray(D)


@jit(nopython = True)
def q_ij(Y):
    """Calculate joint probabilities over all points given Y, the low-dimensional map of data points. (pg. 2585)"""
    
    numerator = np.power(1. + (squared_euc_dist(Y)), -1)
    Q = numerator / np.sum(numerator)
    
    # q_i|i = 0
    np.fill_diagonal(Q, 0.)
    
    return Q

@jit(nopython = True)
def grad_C(P, Q, Y):
    """Estimate the gradient of t-SNE cost with respect to Y."""
    
    pq_diff = np.expand_dims((P - Q), 2)
    
    y_diff = np.expand_dims(Y, 1) - np.expand_dims(Y, 0)
    
    y_dist = np.expand_dims(np.power(1 + squared_euc_dist(Y), -1), 2)

    grad = 4. * (pq_diff * y_diff * y_dist).sum(axis = 1)
    
    return grad


@jit(nopython = True)
def tsne_opt(X, num_iters = 1000, perplexity = 30, alpha = 10, momentum = 0.9):
    """Calculate Y, the optimal low-dimensional representation of data matrix X using optimized TSNE.
    
    Inputs: 
        X: data matrix 
        num_iters: number of iterations 
        perplexity: target perplexity for calculating optimal sigmas for P probability matrix
        alpha: learning rate
        momentum: momentum to speed up gradient descent algorithm 
    """

    # Initialize Y 
    np.random.seed(0)
    Y = np.random.normal(0, 0.0001, size=(X.shape[0], 2))
    D = squared_euc_dist(X)
    P = p_ij(D)
    P = P + np.transpose(P)
    P = P / np.sum(P)

    # Initialise past y_t-1 and y_t-2 values (used for momentum)
    Y_tmin2 = Y
    Y_tmin1 = Y

    # gradient descent with momentum
    for i in range(num_iters):

        Q = q_ij(Y)
        grad = grad_C(P, Q, Y)

        # Update Y using momentum (pg. 2587)
        Y = (Y - (alpha * grad)) +  (momentum * (Y_tmin1 - Y_tmin2))
  
        # update values of y_t-1 and y_t-2
        Y_tmin2 = Y_tmin1
        Y_tmin1 = Y

    return Y



In [68]:
numb_loop = %timeit -o -r3 -n3 tsne_opt(X[:1000, ])

1min 23s ± 583 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


### Try Numba with Looped Distance and Initial PCA

In [61]:
# initialize X by reducing to 50 dimensions using PCA
train = StandardScaler().fit_transform(X[:1000, ])
X_reduce = PCA(n_components=50).fit_transform(train)

In [64]:
@jit(nopython = True)
def p_ij(d_matrix, perplexity = 40.0, tol = 1e-6):
    """
    Finds P_ij matrix using binary search to find value of sigma_i

    Inputs: d_matrix- np.array of pairwise distance matrix, with a fixed perplexity

    Output: P-ij matrix
    """

    (n, d) = d_matrix.shape

    P = np.zeros((n, d), dtype=np.float64)
    prec_sum = 0.0    
    
    # precision = 1/2sigma^2
    for i in range(n):
        prec_min = -np.inf
        prec_max = np.inf
        prec = 1.0

        # implement binary search for optimal sigmas 
        for j in range(10): # 10 binary search steps
            sum_p = 0.0
            for k in range(d):
                if k != i:
                    P[i, k] = np.exp(-d_matrix[i, k] * prec)
                    sum_p += P[i, k]

            sum_p_distribution = 0.0

            for k in range(d):
                P[i, k] /= (sum_p + 1e-8)
                sum_p_distribution += d_matrix[i, k] * P[i, k]

            # Calculate entropy, H matrix
            H = np.log(sum_p) + prec * sum_p_distribution
            H_diff = H - np.log(perplexity)

            
            # check if entropy is within tolerance
            if np.fabs(H_diff) <= tol:
                break

            if H_diff > 0.0:
                prec_min = prec
                if prec_max == np.inf:
                    prec *= 2.0
                else:
                    prec = (prec + prec_max) / 2.0
            else:
                prec_max = prec
                if prec_min == -np.inf:
                    prec /= 2.0
                else:
                    prec = (prec + prec_min) / 2.0

        prec_sum += prec

    return P


@jit(nopython = True)
def squared_euc_dist(X):
    """Calculate squared euclidean distance for all pairs in a data matrix X with d dimensions and n rows. 
    Output is a pairwise distance matrix D that is nxn.
    """
    n = X.shape[0]
    d = X.shape[1]
    D = np.empty((n, n), dtype=np.float64)
    
    for i in range(n):
        for j in range(n):
            dist = 0.0
            for k in range(d):
                diff = X[i, k] - X[j, k]
                dist += diff * diff
            D[i, j] = dist
    return np.asarray(D)


@jit(nopython = True)
def q_ij(Y):
    """Calculate joint probabilities over all points given Y, the low-dimensional map of data points. (pg. 2585)"""
    
    numerator = np.power(1. + (squared_euc_dist(Y)), -1)
    Q = numerator / np.sum(numerator)
    
    # q_i|i = 0
    np.fill_diagonal(Q, 0.)
    
    return Q

@jit(nopython = True)
def grad_C(P, Q, Y):
    """Estimate the gradient of t-SNE cost with respect to Y."""
    
    pq_diff = np.expand_dims((P - Q), 2)
    
    y_diff = np.expand_dims(Y, 1) - np.expand_dims(Y, 0)
    
    y_dist = np.expand_dims(np.power(1 + squared_euc_dist(Y), -1), 2)

    grad = 4. * (pq_diff * y_diff * y_dist).sum(axis = 1)
    
    return grad


@jit(nopython = True)
def tsne_opt(X, num_iters = 1000, perplexity = 40, alpha = 100, momentum = 0.8):
    """Calculate Y, the optimal low-dimensional representation of data matrix X using optimized TSNE.
    
    Inputs: 
        X: data matrix 
        num_iters: number of iterations 
        perplexity: target perplexity for calculating optimal sigmas for P probability matrix
        alpha: learning rate
        momentum: momentum to speed up gradient descent algorithm 
    """

    # Initialize Y 
    np.random.seed(0)
    Y = np.random.normal(0, 0.0001, size=(X.shape[0], 2))
    D = squared_euc_dist(X)
    P = p_ij(D)
    P = P + np.transpose(P)
    P = P / np.sum(P)

    # Initialise past y_t-1 and y_t-2 values (used for momentum)
    Y_tmin2 = Y
    Y_tmin1 = Y

    # gradient descent with momentum
    for i in range(num_iters):

        Q = q_ij(Y)
        grad = grad_C(P, Q, Y)

        # Update Y using momentum (pg. 2587)
        Y = (Y - (alpha * grad)) +  (momentum * (Y_tmin1 - Y_tmin2))
  
        # update values of y_t-1 and y_t-2
        Y_tmin2 = Y_tmin1
        Y_tmin1 = Y

    return Y



In [65]:
numb_loop_pca = %timeit -o -r3 -n3 tsne_opt(X[:1000, ])

1min 22s ± 836 ms per loop (mean ± std. dev. of 3 runs, 3 loops each)


In [71]:
# speed up multiplier

types = [numb, cy, numb_pca, numb_loop, numb_loop_pca]
mult = list(map(lambda x: (normal.average/x.average), types))

import pandas as pd

speed_table_final = pd.DataFrame(mult, index = ['Numba', 'Cython', 'PCA Initialized Numba', 'Numba with Looped Distance', 'PCA Initialized Numba with Looped Distance'], columns = ["Speed-up Multiplier"])
speed_table_final

,Speed-up Multiplier
Numba,4.605476
Cython,0.700209
PCA Initialized Numba,4.992868
Numba with Looped Distance,5.213304
PCA Initialized Numba with Looped Distance,5.291994


In [72]:
speed_table_final.to_csv("speed_table_final.csv")